# Tutorial MongoDB y Scikit-learn

## Instalación/importación de biblioteca pymongo

In [ ]:
#!pip install pymongo
import pymongo

## Conexión, visualización, selección o creación de base de datos
Para comenzar a usar `pymongo`, primero necesitamos establecer una conexión con el servidor de MongoDB. La clase `MongoClient` nos permite crear un objeto de conexión.

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/") # cadena de conexión default para instalacion local
dblist = client.list_database_names()
print(dblist)

## Visualización, selección o creación de la colección
Después de establecer una conexión, podemos seleccionar una db usando el mismo objeto de conexión.

In [ ]:
db = client["datasets_ml"]
print(db.list_collection_names())

Las colecciones en MongoDB son el equivalente a las tablas en bases de datos relacionales. Podemos acceder a las colecciones que residen dentro de una db usando la misma sintaxis tipo diccionario.

In [ ]:
collection = db["iris"]

## Comandos MongoDB
Los comandos `find_one()` y `find()` nos permite realizar consultas y recuperar documentos de una colección basados en un criterio especificado. Es uno de los métodos más fundamentales y comunmente utilizados para obtener datos de una MongoDB. Con él, podemos filtrar documentos, ordenarlos y seleccionar atributos. `find_one()` regresará únicamente un documento que cumpla con los requisitos de la consulta (o `None` si ninguno existe); `find()` regresará un cursor iterable con todos los documentos que cumplan los requisitos.

### Buscar documentos (find)
La sintaxis básica del comando `find()` es la siguiente:
```
documentos = coleccion.find(consulta, proyeccion)
```
- coleccion: La colección en la que deseas buscar documentos.
- consulta: Un diccionario que especifica los criterios de filtro para coincidir con los documentos. Esto es similar a la cláusula WHERE en SQL. Puedes utilizar varios operadores de consulta para expresar condiciones. Si no se especifica una consulta, todos los documentos serán seleccionados.
- proyeccion: Un diccionario que especifica qué campos incluir o excluir de los documentos devueltos. Esto es similar a la cláusula SELECT en SQL. Si no se especifica una proyección, todos los campos serán incluídos.

Dependiendo de cuántos documentos queramos recuperar podemos usar `find_one()` para el primer documento que cumpla con la consulta o `find()` para todos los documentos que cumplan con la consulta.

In [ ]:
doc = collection.find_one()

print(doc)

In [ ]:
for doc in collection.find().limit(5):
  print(doc)

Puedes utilizar el método `limit()` para limitar la cantidad de documentos devueltos, y el método `skip()` para saltar una cierta cantidad de documentos.

In [ ]:
for doc in collection.find().skip(5).limit(5):
  print(doc)

MongoDB utiliza un formato de consulta basado en documentos JSON para definir los criterios de búsqueda y las condiciones que se aplican para recuperar los documentos deseados. Los pares clave-valor se utilizan para describir las propiedades y los valores de los documentos que se están buscando. Adicionalmente, se puede hacer uso de operadores adicionales para definir rangos, pertenencia, diferencia, entre otros. Puedes consultar más al respecto en la [documentación](https://www.mongodb.com/docs/manual/reference/operator/query/).

In [ ]:
query = {'PetalWidthCm': 0.1} # Buscando documentos con PetalWidthCm exactamente igual a 0.1
projection = {'_id': 0, 'SepalLengthCm': 0, 'SepalWidthCm': 0}
for doc in collection.find(query, projection):
    print(doc)

Al especificar una proyección podemos excluir campos (usando `0`) o incluir campos (usando `1`). El campo `_id` siempre se incluye de forma predeterminada a menos que se excluya explícitamente.

In [ ]:
query = {'SepalWidthCm': {'$lt': 2.5}}
projection = {'_id': 0, 'SepalLengthCm': 1, 'SepalWidthCm': 1}
for doc in collection.find(query, projection):
    print(doc)

También puedes utilizar el método `sort()` con el comando `find()` para ordenar los resultados en función de campos específicos.
```
documentos = coleccion.find(consulta).sort(campo, orden)
```
Un argumento `orden` de 1 indica orden ascendente, mientras que -1 indicaría orden descendente.

In [ ]:
query = { "Species": "Iris-versicolor"}
docs = collection.find(query).sort('SepalLengthCm', 1).limit(10)

for d in docs:
  print(d)

### Insertar documentos (insert)
El método `insert_one()` se utiliza para insertar un solo documento en una colección, recibiendo como argumento un diccionario que representa el documento que se desea insertar. Por otro lado, el método `insert_many()` se utiliza para insertar múltiples documentos en una colección, recibiendo como argumento una lista de diccionarios, donde cada diccionario representa un documento a insertar.

Para ambas instrucciones, MongoDB asignará automáticamente un identificador único (`_id`) a cada documento si no se proporciona uno.

In [ ]:
doc = { '_id': 151, 'SepalLengthCm': 5.0, 'SepalWidthCm': 3.2, 'PetalLengthCm': 1.6, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa' }
result = collection.insert_one(doc)
print("Inserted document with id", result.inserted_id)

In [ ]:
query = { "_id": 151}
docs = collection.find(query)

for d in docs:
  print(d)

### Actualizar documentos (update)
Los métodos `update_one()` y `update_many()` permiten actualizar un solo documento o todo documento que cumple con un conjunto específico de criterios de búsqueda en una colección. Así, podemos modificar los valores de los campos en un documento existente de manera selectiva sin afectar otros documentos.
```
update_one(filter, update, upsert=False)
```
- filter: Un diccionario que especifica los criterios de búsqueda para identificar el documento que se debe actualizar. Similar al formato de consulta utilizado en `find()`. El documento que coincida con estos criterios será el que se actualice.
- update: Un diccionario que especifica las modificaciones que se deben realizar en el documento seleccionado. Puedes utilizar operadores de actualización, como `$set`, para modificar campos específicos.
- upsert (opcional): Un valor booleano que indica si se debe insertar un nuevo documento si no se encuentra ningún documento que coincida con los criterios de búsqueda. Si se establece en `True` y no se encuentra ningún documento, se creará uno nuevo con el filtro y la actualización especificados.

In [ ]:
query = { "_id": 151}
newvalues = { "$set": { "SepalLengthCm": 5.1 } }
result = collection.update_one(query, newvalues)
print("Modified count: ", result.modified_count)

### Eliminar documentos (delete)
Los métodos `delete_one()` y `delete_many()` permiten eliminar el primer documento o todo documento que cumple con un conjunto específico de criterios de búsqueda en una colección.

In [ ]:
query = { "_id": 151}
result = collection.delete_one(query)

print("Deleted count:", result.deleted_count)

# Preparación de datos

## MongoDB a DataFrame
El resultado de una consulta a MongoDB puede ser cargado en un `DataFrame` de `pandas` para poder manipularlo más fácilmente. Para hacerlo es necesario primero realizar la consulta usando `pymongo` como se explicó en las secciones previos, obteniendo como resultado un objeto tipo `cursor`.

In [ ]:
import pandas as pd
cursor = collection.find()
print(cursor)
print('Type of cursor:',type(cursor))

Posteriormente, convertir el `cursor` a una lista y proveer dicha lista como argumento a la función `DataFrame` de `pandas`.

In [ ]:
list_cursor = list(cursor)
print(list_cursor[:5])

In [ ]:
df = pd.DataFrame(list_cursor)
print('Type of df:', type(df))

In [ ]:
df.head(5)

## Construcción del conjunto de datos separando variables independientes y la variable dependiente
Con los datos cargados en un `DataFrame` continuamos con la preparación tradicional de los mismos, primero seleccionando y separando atributos de objetivo.

In [ ]:
# Tomamos todas las variables excepto '_id' y 'Species' como variables independiente
X = df.drop(['_id', 'Species'], axis = 1)

# Tomamos solo a la variable 'Species' como variable dependiente
y = df['Species']

# Usamos sklearn para dividir datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

## Creación de un modelo de clasificación utilizando un árbol de decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

## Visualización del árbol de decisión

In [ ]:
#!pip install matplotlib
from sklearn import tree
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(clf, feature_names=list(clf.feature_names_in_), class_names=list(clf.classes_),
               fontsize=10, filled=True, rounded=True)
plt.show()

## Evaluación del modelo

In [ ]:
y_predict = clf.predict(X_test) # Probamos modelo con conjunto de prueba

In [ ]:
from sklearn.metrics import (confusion_matrix,
                           accuracy_score)
from sklearn.metrics import ConfusionMatrixDisplay

# confusion matrix
cm = confusion_matrix(y_test, y_predict)
print ("Confusion Matrix : \n", cm)

# Exactitud de modelo
print('Test accuracy = ', accuracy_score(y_test, y_predict))

## Usando modelo para clasificar nuevos datos
Una vez que el modelo ha sido entrenado y ha cumplido con requisitos de calidad, podemos utilizarlo para clasificar nuevas observaciones que lleguen a la base de datos.

In [ ]:
# Simulamos recolección de nuevas observaciones no clasificadas
docs = [{ '_id': 151, 'SepalLengthCm': 5.0, 'SepalWidthCm': 3.2, 'PetalLengthCm': 1.6, 'PetalWidthCm': 0.2},
        {'_id': 152, 'SepalLengthCm': 5.1, 'SepalWidthCm': 3.1, 'PetalLengthCm': 1.9, 'PetalWidthCm': 0.1},
        {'_id': 153, 'SepalLengthCm': 7, 'SepalWidthCm': 3.2, 'PetalLengthCm': 4.7, 'PetalWidthCm': 1.4}]
result = collection.insert_many(docs)
print("Inserted Ids: ", result.inserted_ids)

Consultando documentos para los cuales no exista un campo `Species` nos permitirá identificar observaciones aún no clasificadas.

In [ ]:
query = {'Species': {'$exists': False}}
docs = collection.find(query)
df = pd.DataFrame(list(docs))
# Seleccionando solo atributos usados para entrenar el modelo
X = df.drop(['_id'], axis=1)
X.head()

Usamos el modelo para generar clases para las nuevas observaciones.

In [ ]:
y_predict = clf.predict(X)
print(y_predict)

In [ ]:
# Agregamos columna con predicciones al DataFrame original
df['Species'] = y_predict
df.head()

Finalmente, actualizamos documentos en la base de datos con la clasificación generada.

In [ ]:
for s in df['Species'].unique(): # Por cada posible clasificacion
    rows = df.loc[df['Species'] == s] # Seleccionamos filas con dicha clasificacion
    filter_query = {'_id': {'$in': rows['_id'].to_list()}} # Generamos filtro para todas las _id de la seleccion
    update_query = {'$set': {'Species': s}} # Agregamos campo 'Species' con el valor correspondiente
    result = collection.update_many(filter_query, update_query) # Actualizamos documentos en la coleccion
    print('Updated', result.modified_count, 'documents with Species', s)

Al finalizar, es recomendable cerrar la conexión con el servidor para liberar recursos.

In [ ]:
client.close()